# RetroReaderMRC

可用的中文预训练参数：[`bert-base`](https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip),[`roberta-wwm-ext-base`](https://drive.google.com/uc?export=download&id=1jMAKIJmPn7kADgD3yQZhpsqM-IRM1qZt),[`roberta-wwm-ext-large`](https://drive.google.com/uc?export=download&id=1dtad0FFzG11CBsawu8hvwwzU2R0FDI94),[`macbert-base`](https://drive.google.com/uc?export=download&id=1aV69OhYzIwj_hn-kO1RiBa-m8QAusQ5b),[`macbert-large`](https://drive.google.com/uc?export=download&id=1lWYxnk1EqTA2Q20_IShxBrCPc5VSDCkT)

In [1]:
import uf

print(uf.__version__)

v2.4.5


In [2]:
model = uf.RetroReaderMRC('../../demo/bert_config.json', '../../demo/vocab.txt')
print(model)

uf.RetroReaderMRC(
    config_file="../../demo/bert_config.json",
    vocab_file="../../demo/vocab.txt",
    max_seq_length=256,
    init_checkpoint=None,
    output_dir=None,
    gpu_ids=None,
    do_lower_case=True,
    reading_module="bert",
    matching_mechanism="cross-attention",
    beta_1=0.5,
    beta_2=0.5,
    threshold=1.0,
    truncate_method="longer-FO",
)


In [3]:
X = [{'doc': '天亮以前说再见，笑着泪流满面。去迎接应该你的，更好的明天', 'ques': '何时说的再见'}, 
     {'doc': '他想知道那是谁，为何总沉默寡言。人群中也算抢眼，抢眼的孤独难免', 'ques': '抢眼的如何'}]
y = [{'text': '天亮以前', 'answer_start': 0}, {'text': '孤独难免', 'answer_start': 27}]

# 训练

In [4]:
model.fit(X, y, total_steps=20)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.4.5-py3.8.egg/uf/model/bert.py:272: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  query_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.4.5-py3.8.egg/uf/model/bert.py:282: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  key_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.4.5-py3.8.egg/uf/model/bert.py:292: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` i

INFO:tensorflow:Build graph with 312,127,500 parameters (among which 104,042,500 are trainable)
INFO:tensorflow:Running local_init_op
INFO:tensorflow:Done running local_init_op
INFO:tensorflow:Running training on 2 samples (step 0 -> 20)
INFO:tensorflow:step 1, has_ans_accuracy 0.0000, exact_match 0.0000, f1 0.0000, sketchy_loss 0.751806, intensive_loss 5.586307, 0.04 steps/sec, 0.08 examples/sec
INFO:tensorflow:step 2, has_ans_accuracy 0.0000, exact_match 0.0000, f1 0.0000, sketchy_loss 0.874139, intensive_loss 5.503387, 0.20 steps/sec, 0.41 examples/sec
INFO:tensorflow:step 3, has_ans_accuracy 0.5000, exact_match 0.0000, f1 0.0000, sketchy_loss 0.128718, intensive_loss 5.516007, 0.21 steps/sec, 0.42 examples/sec
INFO:tensorflow:step 4, has_ans_accuracy 1.0000, exact_match 0.0000, f1 0.0000, sketchy_loss 0.015234, intensive_loss 5.487189, 0.24 steps/sec, 0.48 examples/sec
INFO:tensorflow:step 5, has_ans_accuracy 1.0000, exact_match 0.0000, f1 0.0274, sketchy_loss 0.005343, intensive_l

注：`beta_1`和`beta_2`的取值，决定了sketchy_loss和intensive_loss的下降次序。需要根据收敛情况，回到第[2]步详细地调参

# 推理

In [5]:
model.predict(X)

INFO:tensorflow:Time usage 0m-1.93s, 0.52 steps/sec, 1.04 examples/sec


{'verifier_probs': array([2.964419, 2.944775], dtype=float32),
 'verifier_preds': array([1, 1], dtype=int32),
 'mrc_probs': array([[[0.00391124, 0.00552956, 0.00545779, ..., 0.00313314,
          0.0024408 , 0.00318449],
         [0.00412242, 0.00558457, 0.00615746, ..., 0.00277461,
          0.00287526, 0.00309761]],
 
        [[0.00394706, 0.00458949, 0.0056805 , ..., 0.00312845,
          0.0030584 , 0.00280934],
         [0.0040268 , 0.00410534, 0.00573779, ..., 0.00307305,
          0.00344345, 0.00290215]]], dtype=float32),
 'mrc_preds': ['天亮以前', '，抢眼的孤独难免']}

# 评分

In [6]:
model.score(X, y)

INFO:tensorflow:Time usage 0m-1.19s, 0.84 steps/sec, 1.67 examples/sec


{'has_ans_accuracy': 1.0,
 'exact_match': 0.5,
 'f1': 0.8076916723375778,
 'sketchy_loss': 0.0027561898,
 'intensive_loss': 4.384642}